In [2]:
%load_ext ipyext.writeandexecute

'writeandexecute' magic loaded.


In [20]:
%%writeandexecute -i identifier flatten_json_fields.py
def load_df(csv_path='../data/train.csv', nrows=None):
    import os
    import json
    import pandas as pd
    from pandas.io.json import json_normalize
    
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'},
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = ["{}.{}".format(column, subcolumn) for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print("Loaded {}. Shape: {}".format(os.path.basename(csv_path), df.shape))
    return df

In [3]:
%%writeandexecute -i identifier boolean_to_one_zero.py
# change boolean variables into 1/0 number
def boolean_to_one_zero(df, col, value, bool_value):
    if bool_value:
        df.loc[df[col] == value, col] = 1
        df[col].fillna(0,inplace=True)
    else:
        df.loc[df[col] == value, col] = 0
        df[col].fillna(1,inplace=True)
    df[col] = df[col].astype(int)

In [6]:
%%writeandexecute -i identifier get_time_fields.py
def get_time_fields(x):
    import pandas as pd
    utc = pd.datetime.utcfromtimestamp(x)
    result = {'year': utc.year, 'month' : utc.month,'day': utc.day, 'weekday': utc.weekday(), 'hour': utc.hour}
    return pd.Series(result)